In [3]:
# Jupyter Notebook: Estimating Energetic Content of Sunflower Nectar and Nectar Volume Required for Bee Behaviours

"""
This notebook calculates the energetic content of nectar sugars in sunflower nectar,
and estimates the volume of nectar required to fuel specific bumblebee behaviours (e.g., floral buzzing, takeoff).

References:
- Vear et al. (1990). Genetical studies of nectar and pollen production in sunflower. Agronomie.
- Pattrick et al. (2025). The concentration and energetic content of floral nectar sugars. Journal of Pollination Ecology.
"""

import pandas as pd
import json

# ------------------------------
# SECTION 1: Load behavioural energy data
# ------------------------------

# Load output file from the metabolic analysis script (updated version)
power_df = pd.read_csv("C:/Users/labadmin/Documents/Uppsala analyses/bee_bout_power_output.csv")

# Rename 'flight' to 'takeoff' for consistency
power_df["event_type"] = power_df["event_type"].replace("flight", "takeoff")

# Define event types of interest (must match column 'event_type')
relevant_behaviours = ["buzz", "takeoff"]

# Filter only those behaviours
filtered_df = power_df[power_df["event_type"].isin(relevant_behaviours)].copy()

# ------------------------------
# NEW: Calculate mean energy, duration, power, and nectar per event/second
# ------------------------------

# Map labels just in case
label_map = {"buzz": "buzz", "takeoff": "takeoff"}
filtered_df["behaviour"] = filtered_df["event_type"].map(label_map)

# Behaviour-level means (across events)
beh = (
    filtered_df.groupby("behaviour", as_index=True)
               .agg(mean_energy_J=("Energy_J", "mean"),
                    mean_duration_s=("dt_sec", "mean"),
                    n_events=("Energy_J", "size"))
)
beh["mean_power_W"] = beh["mean_energy_J"] / beh["mean_duration_s"]

# Keep in order
beh = beh.loc[[b for b in ["buzz", "takeoff"] if b in beh.index]]

# ------------------------------
# SECTION 2: Calculate energetic content of nectar
# ------------------------------

# Energetic content (J/mg) from Table 4 in Pattrick et al. (2025)
energy_content = {
    "sucrose": 16.48,
    "glucose": 15.56,
    "fructose": 15.60
}

# Sugar composition of sunflower nectar (Vear et al. 1990)
composition = {
    "fructose": 0.52,
    "glucose": 0.48,
    "sucrose": 0.00
}

# Reported total sugar content (dry matter) = 50% w/w
Cw_combined = 50.0

# Parameters for sucrose solution (used to estimate density of multi-sugar mix)
a1, a2, a3 = 0.998709, 0.0037430, 0.000017639

# Equation 4 from Pattrick et al. (2025): multisugar %w/w to %w/v
def convert_multi_w_w_to_w_v(Cw_sugar, Cw_total, a1, a2, a3):
    density = a1 + a2 * Cw_total + a3 * Cw_total**2
    return Cw_sugar * density

# Compute % w/v for each sugar
Cv_glucose = convert_multi_w_w_to_w_v(Cw_combined * composition["glucose"], Cw_combined, a1, a2, a3)
Cv_fructose = convert_multi_w_w_to_w_v(Cw_combined * composition["fructose"], Cw_combined, a1, a2, a3)
Cv_sucrose  = convert_multi_w_w_to_w_v(Cw_combined * composition["sucrose"], Cw_combined, a1, a2, a3)

# Calculate energetic content of 1 µl of nectar
energy_per_ul = (
    Cv_glucose * energy_content["glucose"] +
    Cv_fructose * energy_content["fructose"] +
    Cv_sucrose  * energy_content["sucrose"]
) / 100  # Convert from J/100 µl to J/µl

# ------------------------------
# Calculate nectar volumes per event and per second for sunflower
# ------------------------------
rows = []
for behaviour_name, row in beh.iterrows():
    rows.append({
        "flower_species": "Helianthus annuus",
        "sugar_percent_w_w": Cw_combined,
        "nectar_energy_J_per_uL": energy_per_ul,
        "behaviour": behaviour_name,
        "mean_energy_J_per_event": row["mean_energy_J"],
        "mean_duration_s_per_event": row["mean_duration_s"],
        "mean_power_W": row["mean_power_W"],
        "nectar_uL_per_event": row["mean_energy_J"] / energy_per_ul,
        "nectar_uL_per_s": row["mean_power_W"] / energy_per_ul,
        "n_events": row["n_events"],
    })

sunflower_df = pd.DataFrame(rows)

# Round for presentation
round_cols = ["mean_energy_J_per_event", "mean_duration_s_per_event",
              "mean_power_W", "nectar_uL_per_event", "nectar_uL_per_s"]
sunflower_df[round_cols] = sunflower_df[round_cols].round(4)

display(sunflower_df)

# ------------------------------
# Suggested legend text
# ------------------------------
legend_bits = []
if "buzz" in beh.index:
    legend_bits.append(
        f"buzz: {beh.loc['buzz','mean_energy_J']:.4f} J; {beh.loc['buzz','mean_duration_s']:.2f} s"
    )
if "takeoff" in beh.index:
    legend_bits.append(
        f"take-off: {beh.loc['takeoff','mean_energy_J']:.4f} J; {beh.loc['takeoff','mean_duration_s']:.2f} s"
    )

legend_text = (
    "Behavioural energy costs and durations (means across events) were applied uniformly "
    "across species. "
    f"For sunflower: {', '.join(legend_bits)}. "
    "Nectar volumes are reported per event and per second (µL·s⁻¹)."
)
print("\nLegend:\n" + legend_text)

,flower_species,sugar_percent_w_w,nectar_energy_J_per_uL,behaviour,mean_energy_J_per_event,mean_duration_s_per_event,mean_power_W,nectar_uL_per_event,nectar_uL_per_s,n_events
0,Helianthus annuus,50.0,9.581853,buzz,0.0992,1.5923,0.0623,0.0104,0.0065,260.0
1,Helianthus annuus,50.0,9.581853,takeoff,0.0968,1.4225,0.0681,0.0101,0.0071,213.0



Legend:
Behavioural energy costs and durations (means across events) were applied uniformly across species. For sunflower: buzz: 0.0992 J; 1.59 s, take-off: 0.0968 J; 1.42 s. Nectar volumes are reported per event and per second (µL·s⁻¹).


In [4]:
# Jupyter Notebook: Estimated Nectar Volume for Bee Behaviours — Rubus idaeus

import pandas as pd
import json

# ------------------------------
# SECTION 1: Load behavioural energy data
# ------------------------------
# If you've already loaded this above, you can comment the next line.
power_df = pd.read_csv("C:/Users/labadmin/Documents/Uppsala analyses/bee_bout_power_output.csv")

# Rename 'flight' to 'takeoff' for consistency
power_df["event_type"] = power_df["event_type"].replace("flight", "takeoff")

# Events of interest
relevant_behaviours = ["buzz", "takeoff"]
filtered_df = power_df[power_df["event_type"].isin(relevant_behaviours)].copy()

# ------------------------------
# NEW: Calculate mean energy, duration, power (per event)
# ------------------------------
filtered_df["behaviour"] = filtered_df["event_type"]

beh = (
    filtered_df.groupby("behaviour", as_index=True)
               .agg(mean_energy_J=("Energy_J", "mean"),      # J per event
                    mean_duration_s=("dt_sec", "mean"),      # s per event
                    n_events=("Energy_J", "size"))
)
beh["mean_power_W"] = beh["mean_energy_J"] / beh["mean_duration_s"]
beh = beh.loc[[b for b in ["buzz", "takeoff"] if b in beh.index]]

# ------------------------------
# SECTION 2: Nectar energy (Rubus idaeus)
# ------------------------------
# From your table:
Cw_combined = 44.6                      # % w/w total sugars
energy_per_ul = 8.37                    # J/µL (Rubus idaeus; from Kostryco & Chwil, 2022 + your calc)

# ------------------------------
# NEW: Nectar volumes per event and per second (raspberry)
# ------------------------------
rows = []
for behaviour_name, row in beh.iterrows():
    rows.append({
        "flower_species": "Rubus idaeus",
        "sugar_percent_w_w": Cw_combined,
        "nectar_energy_J_per_uL": energy_per_ul,
        "behaviour": behaviour_name,
        "mean_energy_J_per_event": row["mean_energy_J"],
        "mean_duration_s_per_event": row["mean_duration_s"],
        "mean_power_W": row["mean_power_W"],
        "nectar_uL_per_event": row["mean_energy_J"] / energy_per_ul,
        "nectar_uL_per_s": row["mean_power_W"] / energy_per_ul,
        "n_events": row["n_events"],
    })

raspberry_df = pd.DataFrame(rows)

# Round for presentation
round_cols = ["mean_energy_J_per_event", "mean_duration_s_per_event",
              "mean_power_W", "nectar_uL_per_event", "nectar_uL_per_s"]
raspberry_df[round_cols] = raspberry_df[round_cols].round(4)

display(raspberry_df)

# ------------------------------
# Suggested legend text (raspberry)
# ------------------------------
legend_bits = []
if "buzz" in beh.index:
    legend_bits.append(
        f"buzz: {beh.loc['buzz','mean_energy_J']:.4f} J; {beh.loc['buzz','mean_duration_s']:.2f} s"
    )
if "takeoff" in beh.index:
    legend_bits.append(
        f"take-off: {beh.loc['takeoff','mean_energy_J']:.4f} J; {beh.loc['takeoff','mean_duration_s']:.2f} s"
    )

legend_text = (
    "Behavioural energy costs and durations (means across events) were applied uniformly across species. "
    f"For raspberry: {', '.join(legend_bits)}. "
    "Nectar volumes are reported per event and per second (µL·s⁻¹)."
)
print("\nLegend:\n" + legend_text)

,flower_species,sugar_percent_w_w,nectar_energy_J_per_uL,behaviour,mean_energy_J_per_event,mean_duration_s_per_event,mean_power_W,nectar_uL_per_event,nectar_uL_per_s,n_events
0,Rubus idaeus,44.6,8.37,buzz,0.0992,1.5923,0.0623,0.0119,0.0074,260.0
1,Rubus idaeus,44.6,8.37,takeoff,0.0968,1.4225,0.0681,0.0116,0.0081,213.0



Legend:
Behavioural energy costs and durations (means across events) were applied uniformly across species. For raspberry: buzz: 0.0992 J; 1.59 s, take-off: 0.0968 J; 1.42 s. Nectar volumes are reported per event and per second (µL·s⁻¹).


In [5]:
# Jupyter Notebook: Estimated Nectar Volume for Bee Behaviours — Borago officinalis

import pandas as pd
import json

# ------------------------------
# SECTION 1: Load behavioural energy data
# ------------------------------
# If you've already loaded this above, you can comment the next line.
power_df = pd.read_csv("C:/Users/labadmin/Documents/Uppsala analyses/bee_bout_power_output.csv")

# Rename 'flight' to 'takeoff' for consistency
power_df["event_type"] = power_df["event_type"].replace("flight", "takeoff")

# Events of interest
relevant_behaviours = ["buzz", "takeoff"]
filtered_df = power_df[power_df["event_type"].isin(relevant_behaviours)].copy()

# ------------------------------
# NEW: Calculate mean energy, duration, power (per event)
# ------------------------------
filtered_df["behaviour"] = filtered_df["event_type"]

beh = (
    filtered_df.groupby("behaviour", as_index=True)
               .agg(mean_energy_J=("Energy_J", "mean"),      # J per event
                    mean_duration_s=("dt_sec", "mean"),      # s per event
                    n_events=("Energy_J", "size"))
)
beh["mean_power_W"] = beh["mean_energy_J"] / beh["mean_duration_s"]
beh = beh.loc[[b for b in ["buzz", "takeoff"] if b in beh.index]]

# ------------------------------
# SECTION 2: Nectar energy (Borago officinalis)
# ------------------------------
# From your table:
Cw_combined = 60.0                       # % w/w total sugars
energy_per_ul = 12.40                    # J/µL (Borago officinalis; from Descamps et al., 2021 + your calc)

# ------------------------------
# NEW: Nectar volumes per event and per second (borage)
# ------------------------------
rows = []
for behaviour_name, row in beh.iterrows():
    rows.append({
        "flower_species": "Borago officinalis",
        "sugar_percent_w_w": Cw_combined,
        "nectar_energy_J_per_uL": energy_per_ul,
        "behaviour": behaviour_name,
        "mean_energy_J_per_event": row["mean_energy_J"],
        "mean_duration_s_per_event": row["mean_duration_s"],
        "mean_power_W": row["mean_power_W"],
        "nectar_uL_per_event": row["mean_energy_J"] / energy_per_ul,
        "nectar_uL_per_s": row["mean_power_W"] / energy_per_ul,
        "n_events": row["n_events"],
    })

borage_df = pd.DataFrame(rows)

# Round for presentation
round_cols = ["mean_energy_J_per_event", "mean_duration_s_per_event",
              "mean_power_W", "nectar_uL_per_event", "nectar_uL_per_s"]
borage_df[round_cols] = borage_df[round_cols].round(4)

display(borage_df)

# ------------------------------
# Suggested legend text (borage)
# ------------------------------
legend_bits = []
if "buzz" in beh.index:
    legend_bits.append(
        f"buzz: {beh.loc['buzz','mean_energy_J']:.4f} J; {beh.loc['buzz','mean_duration_s']:.2f} s"
    )
if "takeoff" in beh.index:
    legend_bits.append(
        f"take-off: {beh.loc['takeoff','mean_energy_J']:.4f} J; {beh.loc['takeoff','mean_duration_s']:.2f} s"
    )

legend_text = (
    "Behavioural energy costs and durations (means across events) were applied uniformly across species. "
    f"For borage: {', '.join(legend_bits)}. "
    "Nectar volumes are reported per event and per second (µL·s⁻¹)."
)
print("\nLegend:\n" + legend_text)

,flower_species,sugar_percent_w_w,nectar_energy_J_per_uL,behaviour,mean_energy_J_per_event,mean_duration_s_per_event,mean_power_W,nectar_uL_per_event,nectar_uL_per_s,n_events
0,Borago officinalis,60.0,12.4,buzz,0.0992,1.5923,0.0623,0.0080,0.0050,260.0
1,Borago officinalis,60.0,12.4,takeoff,0.0968,1.4225,0.0681,0.0078,0.0055,213.0



Legend:
Behavioural energy costs and durations (means across events) were applied uniformly across species. For borage: buzz: 0.0992 J; 1.59 s, take-off: 0.0968 J; 1.42 s. Nectar volumes are reported per event and per second (µL·s⁻¹).
